In [12]:
import logging, logging.config
logging.config.fileConfig('../config/logging.conf')
logger = logging.getLogger(__name__)

import os
import sys
from pathlib import Path
import pickle

rootdir = Path().resolve()
sys.path.insert(0, os.path.abspath(os.path.join(rootdir , '../source')))
sys.path.insert(0, os.path.abspath(os.path.join(rootdir , '../config')))

from setting_freq import *

import data_make 
import matplotlib as mpl
mpl.use('Agg')

import numpy as np
import matplotlib.pyplot as plt
import solver_mfreq as s_freq
import plot_make 


def main():


    lambda_arr = np.array([1.0, 1.3])
    lambda0 = 1.0
    nu_arr = 1/lambda_arr
    nu0 = 1/lambda0

    #Making images
    input_model, xx, yy = data_make.ring_make_multi_frequency(XNUM, YNUM, DX, DY, \
        RAD_RING, WIDTH_RING, nu_arr, nu0, None, function = data_make.gaussian_function_1d)
    print(np.shape(input_model))

    #"""
    ## Vertual Obervatory
    obs_name = "test_observatory_mfreq"
    obs_file = obs_name + ".pk"
    if not os.path.exists(obs_file) or REPLACE_OBS:
        obs_ex = data_make.observatory_mfreq(input_model, NDATA , PERIOD, SN , OBS_DUR  , N_ANTE ,BASELINE_UVMAX, [0., 0], lambda_arr, save_folder = FIG_FOLDER)
        obs_ex.set_antn()
        obs_ex.plotter_uv_sampling()

        with open(obs_file, "wb") as f:
            pickle.dump(obs_ex, f)

    else:
        with open(obs_file, "rb") as f:
            obs_ex = pickle.load(f) 

        ## Making obs data
    vis_obs, num_mat, fft_now = obs_ex.obs_make(DX, DY, SN)
    vis_test = vis_obs

    ## Images from obs
    """
    beam_func = np.zeros(np.shape(num_mat))
    beam_func[num_mat>0] = 1
    beam_func = np.fft.fftshift(beam_func)
    dirty_image = np.fft.ifft2(vis_obs)
    beam_image = np.fft.fftshift(np.fft.ifft2(beam_func))
    plot_make.image_2dplot(np.abs(beam_image),  lim = 127)
    """


    ## Setting priors for model
    N_tot = XNUM * YNUM
    model_init = np.zeros(2 * N_tot)
    model_init[N_tot:2*N_tot] = 2
    ##
    bounds = []
    for i in range(N_tot):
        bounds.append([-1e-3,1000])

    for i in range(N_tot):
        bounds.append([-10,10])

    bounds = np.array(bounds)
    f_cost= s_freq.multi_freq_cost_l1_tsv
    df_cost = s_freq.multi_freq_grad
    lambda_l1 = 1e0
    lambda_ltsv = 1e0
    args = (vis_obs, nu_arr, nu0, lambda_l1, lambda_ltsv)
    print(s_freq.multi_freq_cost_l1_tsv(model_init,vis_obs, nu_arr, nu0, lambda_l1, lambda_ltsv))
    result = s_freq.solver_mfreq(f_cost,df_cost, model_init, bounds,  vis_obs, nu_arr, nu0, lambda_l1, lambda_ltsv) 
    print(result)

if __name__ == "__main__":
    logger.info("Main part starts")
    main()

(2, 64, 64)
(2, 2, 2700)
2 64 64
fft_shape: (2, 64, 64)
(2, 2, 2700)
(2, 2, 2700)
119604526.98100832
(array([0., 0., 0., ..., 2., 2., 2.]), 119604526.98100832, {'grad': array([-8.72652867e+03, -7.68607712e+03, -6.08801839e+03, ...,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 21, 'nit': 0, 'warnflag': 2})
